<a href="https://colab.research.google.com/github/Domqwerty/whisper-subtitle-generator/blob/main/generate_subtitles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installa Whisper e FFmpeg
!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import whisper

# Parametro per la formattazione dei sottotitoli
MAX_WORDS_PER_LINE = 9  # Numero massimo di parole per riga

# Carica il modello Whisper
model = whisper.load_model("small")  # Usa "tiny" se vuoi maggiore velocità

# Trascrivi l'audio specificando la lingua
# Sostituisci "/content/input.mp3" con il percorso del tuo file audio
result = model.transcribe("/content/input.mp3", language="en", word_timestamps=True)

# Funzione per dividere il testo in linee basate su max_words_per_line
def split_text_into_lines_with_timestamps(words, max_words_per_line):
    lines = []
    current_line = []
    current_start = None
    current_end = None

    for i in range(0, len(words), max_words_per_line):
        # Prendi un blocco di parole
        block = words[i:i + max_words_per_line]

        # Calcola il timestamp iniziale e finale per il blocco
        start_time = block[0]["start"]
        end_time = block[-1]["end"]

        # Unisci le parole del blocco in una riga di testo
        line_text = " ".join([word["word"].strip() for word in block])

        # Aggiungi la riga alla lista
        lines.append({
            "text": line_text,
            "start": start_time,
            "end": end_time
        })

    return lines

# Funzione per formattare il tempo nel formato SRT
def format_time(seconds):
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = int(seconds % 60)
    millis = int((seconds - int(seconds)) * 1000)
    return f"{hours:02}:{minutes:02}:{secs:02},{millis:03}"

# Salva il risultato in un file SRT
output_file = "/content/output.srt"
with open(output_file, "w", encoding="utf-8") as file:
    index = 1
    for segment in result["segments"]:
        # Ottieni le parole con i loro timestamp
        words = segment["words"]

        # Dividi il testo in linee basate su max_words_per_line
        lines = split_text_into_lines_with_timestamps(words, MAX_WORDS_PER_LINE)

        # Scrivi ciascuna linea come un blocco SRT separato
        for line in lines:
            file.write(f"{index}\n")
            file.write(f"{format_time(line['start'])} --> {format_time(line['end'])}\n")
            file.write(f"{line['text']}\n\n")
            index += 1

print(f"Trascrizione completata! Il file SRT è stato salvato come '{output_file}'.")